In [1]:
# Using GLM to calculate Y_fit_train and Y_fit_test
import os
import glob
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# === FUNCTIONS ===

def load_data(pkl_path, excel_path):
    X = pd.read_pickle(pkl_path)
    Y = pd.read_excel(excel_path).set_index('DateTime')['Prec_23.0_72.5']
    return X, Y

def split_data(X, Y, split_index=3287):
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    Y_train, Y_test = Y.iloc[:split_index], Y.iloc[split_index:]
    return X_train, X_test, Y_train, Y_test

def binary_glm(X_train, Y_train):
    yb_train = (Y_train > 0).astype(int)
    glm = sm.GLM(yb_train, sm.add_constant(X_train), family=sm.families.Binomial()).fit()
    yfit_train = glm.predict(sm.add_constant(X_train))
    return yfit_train

def quantile_regression(X, Y, q):
    mod = sm.QuantReg(Y, sm.add_constant(X)).fit(q=q)
    return mod.params

def reconstruct_rain(X, coefs):
    return sm.add_constant(X) @ coefs

def cqm_pipeline(X_train, Y_train, X_test, thresholds):
    yfit_train = binary_glm(X_train, Y_train)

    coefs2 = {}
    rain_final = {}

    for label, t in thresholds.items():
        q_val = float(label.replace('p','')) / 100

        mask1 = yfit_train > t
        X1, Y1 = X_train[mask1], Y_train[mask1]
        mod1 = quantile_regression(X1, Y1, q_val)
        rain1 = reconstruct_rain(X1, mod1)

        mask2 = rain1 > 0
        X2, Y2 = X1[mask2], Y1[mask2]
        mod2 = quantile_regression(X2, Y2, q_val)
        coefs2[label] = mod2
        rain_test = reconstruct_rain(X_test, mod2)
        rain_final[label] = np.maximum(rain_test, 0)

    return coefs2, rain_final

def extract_jjas(series):
    series.index = pd.to_datetime(series.index)
    return series[series.index.month.isin([6,7,8,9])]

def compute_cqvss(y_true, y_pred_model, y_pred_ref, q):
    errors_m = y_true - y_pred_model
    errors_r = y_true - y_pred_ref
    ql_m = np.mean(np.maximum(q * errors_m, (q - 1) * errors_m))
    ql_r = np.mean(np.maximum(q * errors_r, (q - 1) * errors_r))
    return 1 - (ql_m / ql_r) if ql_r != 0 else np.nan

# === CONFIG ===

pca_folder = r'D:/D/Ruvision/ERA5/Pickle format/PCA/Logistic_reg'
obs_excel = r'C:/Users/Angshudeep Majumdar/Documents/Angshudeep Lappy/Ruvision/IMD_2015-2024_Daily Data_0.25 resolution Rain.xlsx'
output_base = r'D:/D/Ruvision/plots/ERA/GLM'

thresholds = {'80p': 0.2, '85p': 0.15, '90p': 0.1, '95p': 0.05, '99p': 0.01}
quantiles = [0.8, 0.85, 0.9, 0.95, 0.99]
all_conf_matrices=[]

# === MAIN LOOP ===

for pkl in glob.glob(os.path.join(pca_folder, '*.pkl')):
    var_comb = os.path.basename(pkl).split('transformed_dataX_')[1].split('.pkl')[0]
    out_dir = os.path.join(output_base, var_comb)
    os.makedirs(out_dir, exist_ok=True)

    # Load & split
    X, Y = load_data(pkl, obs_excel)
    X_train, X_test, Y_train, Y_test = split_data(X, Y)

    # Run modular 2-step CQM
    coefs2, rain_final = cqm_pipeline(X_train, Y_train, X_test, thresholds)

    # Run SQM
    sqm_models = []
    sqm_final = {}
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q)
        sqm_models.append(mod)
        sqm_final[f"{int(q*100)}p"] = np.maximum(mod.predict(sm.add_constant(X_test)), 0)

    # JJAS
    Y_test.index = pd.to_datetime(Y_test.index)
    mask_jjas = Y_test.index.month.isin([6,7,8,9])
    Y_jjas = Y_test[mask_jjas]
    X_test_jjas = X_test[mask_jjas]

    rain_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in rain_final.items()}
    sqm_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in sqm_final.items()}

    # Linear Regression
    mask_train_jjas = Y_train.index.month.isin([6,7,8,9])
    lr = LinearRegression().fit(X_train[mask_train_jjas], Y_train[mask_train_jjas])
    lr_pred = lr.predict(X_test_jjas)

    # === CONTINGENCY + METRICS ===

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas)
    y_pred_ref = lr_pred

    results = []
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final_jjas: continue

        y_pred_cqm = rain_final_jjas[q_label].values
        y_pred_sqm = sqm_final_jjas[q_label].values

    # === CORRECT SQM ===
    sqm_models = []
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q, max_iter=5000)
        sqm_models.append(mod)

    sqm_final = {}
    for i, q in enumerate(quantiles):
        rain_q = sqm_models[i].predict(sm.add_constant(X_test))
        rain_q = np.maximum(rain_q, 0)
        sqm_final[f"{int(q*100)}p"] = pd.Series(rain_q, index=X_test.index)

    # === Filter SQM for JJAS ===
    sqm_final_jjas = {}
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label in sqm_final:
            sqm_final_jjas[q_label] = sqm_final[q_label][sqm_final[q_label].index.month.isin([6,7,8,9])]

    # === Train/Test JJAS split ===
    train_index = Y_train.index
    mask_train_jjas = train_index.month.isin([6, 7, 8, 9])
    Y_train_jjas = Y_train[mask_train_jjas]
    X_train_jjas = X_train[mask_train_jjas]
    test_index = Y_test.index
    mask_test_jjas = test_index.month.isin([6, 7, 8, 9])
    X_test_jjas = X_test[mask_test_jjas]
    Y_jjas = Y_test[mask_test_jjas]

    # === Linear Regression ===
    lr = LinearRegression().fit(X_train_jjas, Y_train_jjas)
    pred_lr_jjas = lr.predict(X_test_jjas)
    pred_lr_jjas = pd.Series(pred_lr_jjas, index=X_test_jjas.index)

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas).flatten()
    y_pred_ref = np.array(pred_lr_jjas).flatten()

    results = []
    reports=[]
    for q in quantiles:
        q_label = f"{int(q*100)}p"

        if q_label not in rain_final or q_label not in sqm_final_jjas:
            continue

        y_pred_cqm = np.array(pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]).flatten()
        y_pred_sqm = np.array(sqm_final_jjas[q_label]).flatten()

        obs_bin = y_true > threshold_value
        pred_bin_cqm = y_pred_cqm > threshold_value
        pred_bin_sqm = y_pred_sqm > threshold_value
        pred_bin_lr = y_pred_ref > threshold_value

        H = np.sum((obs_bin == 1) & (pred_bin_cqm == 1))
        M = np.sum((obs_bin == 1) & (pred_bin_cqm == 0))
        FA = np.sum((obs_bin == 0) & (pred_bin_cqm == 1))
        CN = np.sum((obs_bin == 0) & (pred_bin_cqm == 0))

        POD = H / (H + M) if (H + M) > 0 else np.nan
        FAR = FA / (H + FA) if (H + FA) > 0 else np.nan
        Accuracy = (H + CN) / (H + M + FA + CN) if (H + M + FA + CN) > 0 else np.nan
        Precision = H / (H + FA) if (H + FA) > 0 else np.nan
        F1 = 2 * (Precision * POD) / (Precision + POD) if (Precision + POD) > 0 else np.nan
        TS = H / (H + M + FA) if (H + M + FA) > 0 else np.nan
        BIAS = (H + FA) / (H + M) if (H + M) > 0 else np.nan
        denom = ((H + M) * (M + CN)) + ((H + FA) * (FA + CN))
        HSS = (2 * (H * CN - FA * M)) / denom if denom != 0 else np.nan
        corr, _ = pearsonr(y_true, y_pred_cqm)
        cqvss = compute_cqvss(y_true, y_pred_cqm, y_pred_ref, q)

        H_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 1))
        M_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 0))
        FA_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 1))
        CN_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 0))
        POD_LR = H_LR / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        FAR_LR = FA_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        Accuracy_LR = (H_LR + CN_LR) / (H_LR + M_LR + FA_LR + CN_LR) if (H_LR + M_LR + FA_LR + CN_LR) > 0 else np.nan
        Precision_LR = H_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        F1_LR = 2 * (Precision_LR * H_LR) / (Precision_LR + H_LR) if (Precision_LR + H_LR) > 0 else np.nan
        TS_LR = H_LR / (H_LR + M_LR + FA_LR) if (H_LR + M_LR + FA_LR) > 0 else np.nan
        BIAS_LR = (H_LR + FA_LR) / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        corr_LR, _ = pearsonr(y_true, y_pred_ref)
        
        H_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 1))
        M_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 0))
        FA_SQM = np.sum((obs_bin == 0) & (pred_bin_sqm == 1))
        CN_SQM= np.sum((obs_bin == 0) & (pred_bin_sqm == 0))
        POD_SQM = H_SQM / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        FAR_SQM = FA_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        Accuracy_SQM = (H_SQM + CN_SQM) / (H_SQM + M_SQM + FA_SQM + CN_SQM) if (H_SQM + M_SQM + FA_SQM + CN_SQM) > 0 else np.nan
        Precision_SQM = H_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        F1_SQM = 2 * (Precision_SQM * H_SQM) / (Precision_SQM + H_SQM) if (Precision_SQM + H_SQM) > 0 else np.nan
        TS_SQM = H_SQM / (H_SQM + M_SQM + FA_SQM) if (H_SQM + M_SQM + FA_SQM) > 0 else np.nan
        BIAS_SQM = (H_SQM + FA_SQM) / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        denom_SQM = ((H_SQM + M_SQM) * (M_SQM + CN_SQM)) + ((H_SQM + FA_SQM) * (FA_SQM + CN_SQM))
        HSS_SQM = (2 * (H_SQM * CN_SQM - FA_SQM * M_SQM)) / denom_SQM if denom_SQM != 0 else np.nan
        corr_sqm, _ = pearsonr(y_true, y_pred_sqm)
        cqvss_sqm = compute_cqvss(y_true, y_pred_sqm, y_pred_ref, q)

        results.append({
            'Quantile': q,
            'Hits_CQM': H, 'Misses_CQM': M, 'False Alarms_CQM': FA, 'Correct Negatives_CQM': CN,
            'Correlation_CQM': round(corr, 3),
            'Hit Rate_CQM': round(POD, 3),
            'False Alarm Ratio_CQM': round(FAR, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'Precision_CQM': round(Precision, 3),
            'F1 Score_CQM': round(F1, 3),
            'Threat Score_CQM': round(TS, 3),
            'Bias Score_CQM': round(BIAS, 3),
            'Heidke Skill Score_CQM': round(HSS, 3),
            'CQVSS_CQM': round(cqvss, 3),
            'Hit Rate_LR': round(POD_LR, 3),
            'False Alarm Ratio_LR': round(FAR_LR, 3),
            'Accuracy_LR': round(Accuracy_LR, 3),
            'Precision_LR': round(Precision_LR, 3),
            'F1 Score_LR': round(F1_LR, 3),
            'Threat Score_LR': round(TS_LR, 3),
            'Bias_LR': round(BIAS_LR, 3),
            'Correlation_LR':round(corr_LR,3),
            'Hits_SQM': H_SQM, 'Misses_SQM': M_SQM, 'False Alarms_SQM': FA_SQM,
            'Correlation_SQM': round(corr_sqm, 3),
            'Hit Rate_SQM': round(POD_SQM, 3),
            'False Alarm Ratio_SQM': round(FAR_SQM, 3),
            'Accuracy_SQM': round(Accuracy_SQM, 3),
            'Precision_SQM': round(Precision_SQM, 3),
            'F1 Score_SQM': round(F1_SQM, 3),
            'Threat Score_SQM': round(TS_SQM, 3),
            'Bias_SQM': round(BIAS_SQM, 3),
            'Heidke Skill Score_SQM': round(HSS_SQM, 3),
            'SQVSS_SQM': round(cqvss_sqm, 3)
        })
        reports.append({
            'Quantile': q,
            'Hits_CQM': H,
            'Misses_CQM': M,
            'False Alarms_CQM': FA,
            'Correct Negatives_CQM': CN,
            'Recall_CQM': round(POD, 3),
            'Precision_CQM': round(Precision, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'F1 Score_CQM': round(F1, 3)
        })
    
    pd.DataFrame(results).to_excel(os.path.join(out_dir, 'contingency_matrix.xlsx'), index=False)

    conf_matrix = pd.DataFrame([
        {
            'Variable_Combination': var_comb,
            **r
        } for r in reports
    ])
    all_conf_matrices.append(conf_matrix)

    # === SAVE FINAL CONCATENATED CONFUSION MATRIX ===

    if all_conf_matrices:
        final_df = pd.concat(all_conf_matrices, ignore_index=True)
        final_df.to_excel(r'C:\Users\Angshudeep Majumdar\Downloads\all_confusion_matrices.xlsx', index=False)

    # === Plots stay same, but now use sqm_final_jjas ===

    # === CQM Plots ===
    fig, axs = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final: continue
        cqm_jjas = pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]
        r, _ = pearsonr(Y_jjas, cqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, cqm_jjas))
        axs[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs[i].plot(cqm_jjas.index, cqm_jjas, label=f'CQM Q{q}', color='red')
        axs[i].set_title(f'CQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'GLM based CQR Model Experiment | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(os.path.join(out_dir, f'CQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig)

    # === SQM Plots ===
    fig2, axs2 = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in sqm_final_jjas: continue
        sqm_jjas = sqm_final_jjas[q_label]
        r, _ = pearsonr(Y_jjas, sqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, sqm_jjas))
        axs2[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs2[i].plot(sqm_jjas.index, sqm_jjas, label=f'SQM Q{q}', color='green')
        axs2[i].set_title(f'SQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs2[i].tick_params(axis='x', rotation=45)
        axs2[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'SQM Prediction | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig2.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig2.tight_layout(rect=[0, 0, 1, 0.95])
    fig2.savefig(os.path.join(out_dir, f'SQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig2)

    # === Skill Score Plots ===
    metrics_df = pd.DataFrame(results)
    x_labels = [f"{int(q)}p" for q in metrics_df['Quantile'] * 100]

    fig3, axs3 = plt.subplots(1, 6, figsize=(30, 6))
    axs3[0].bar(x_labels, metrics_df['Hit Rate_CQM'], color='red'); axs3[0].set_title('Hit Rate (POD)')
    axs3[1].bar(x_labels, metrics_df['False Alarm Ratio_CQM'], color='orange'); axs3[1].set_title('False Alarm Ratio (FAR)')
    axs3[2].bar(x_labels, metrics_df['Threat Score_CQM'], color='blue'); axs3[2].set_title('Threat Score (TS)')
    axs3[3].bar(x_labels, metrics_df['Bias Score_CQM'], color='purple'); axs3[3].set_title('Bias Score')
    axs3[4].bar(x_labels, metrics_df['Heidke Skill Score_CQM'], color='green'); axs3[4].set_title('HSS')
    axs3[5].bar(x_labels, metrics_df['CQVSS_CQM'], color='pink'); axs3[5].set_title('CQVSS')
    fig3.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_GLM based CQR Model Experiment', fontsize=16)
    fig3.tight_layout(rect=[0, 0, 1, 0.95])
    fig3.savefig(os.path.join(out_dir, f'CQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig3)

    fig4, axs4 = plt.subplots(1, 6, figsize=(30, 6))
    axs4[0].bar(x_labels, metrics_df['Hit Rate_SQM'], color='red'); axs4[0].set_title('Hit Rate (POD)')
    axs4[1].bar(x_labels, metrics_df['False Alarm Ratio_SQM'], color='orange'); axs4[1].set_title('False Alarm Ratio (FAR)')
    axs4[2].bar(x_labels, metrics_df['Threat Score_SQM'], color='blue'); axs4[2].set_title('Threat Score (TS)')
    axs4[3].bar(x_labels, metrics_df['Bias_SQM'], color='purple'); axs4[3].set_title('Bias Score')
    axs4[4].bar(x_labels, metrics_df['Heidke Skill Score_SQM'], color='green'); axs4[4].set_title('HSS')
    axs4[5].bar(x_labels, metrics_df['SQVSS_SQM'], color='pink'); axs4[5].set_title('SQVSS')
    fig4.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_SQM', fontsize=16)
    fig4.tight_layout(rect=[0, 0, 1, 0.95])
    fig4.savefig(os.path.join(out_dir, f'SQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig4)

    # === Linear Regression ===
    fig5, ax5 = plt.subplots(figsize=(15, 6))
    r_lr, _ = pearsonr(Y_jjas, pred_lr_jjas)
    rmse_lr = np.sqrt(mean_squared_error(Y_jjas, pred_lr_jjas))
    ax5.plot(Y_jjas.index, Y_jjas, label='Observed', color='blue')
    ax5.plot(pred_lr_jjas.index, pred_lr_jjas, label='Linear Regression', color='black')
    ax5.set_title(f'Linear Regression | {var_comb} | JJAS | r={r_lr:.2f} | RMSE={rmse_lr:.2f}', fontsize=14)
    ax5.tick_params(axis='x', rotation=45)
    ax5.legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'Linear Regression | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig5.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig5.tight_layout(rect=[0, 0, 1, 0.95])
    fig5.savefig(os.path.join(out_dir, f'LinearRegression_{var_comb}_JJAS.png'))
    plt.close(fig5)


C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar

In [2]:
# Using Logistic Regression to calculate Y_fit_train and Y_fit_test
import os
import glob
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# === FUNCTIONS ===

def load_data(pkl_path, excel_path):
    X = pd.read_pickle(pkl_path)
    Y = pd.read_excel(excel_path).set_index('DateTime')['Prec_23.0_72.5']
    return X, Y

def split_data(X, Y, split_index=3287):
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    Y_train, Y_test = Y.iloc[:split_index], Y.iloc[split_index:]
    return X_train, X_test, Y_train, Y_test

def binary_logistic(X_train, Y_train):
    yb_train = (Y_train > 0).astype(int)
    logreg = LogisticRegression(max_iter=5000).fit(X_train, yb_train)
    yfit_train = logreg.predict_proba(X_train)[:, 1]
    return yfit_train

def quantile_regression(X, Y, q):
    mod = sm.QuantReg(Y, sm.add_constant(X)).fit(q=q)
    return mod.params

def reconstruct_rain(X, coefs):
    return sm.add_constant(X) @ coefs

def cqm_pipeline(X_train, Y_train, X_test, thresholds):
    yfit_train = binary_logistic(X_train, Y_train)

    coefs2 = {}
    rain_final = {}

    for label, t in thresholds.items():
        q_val = float(label.replace('p','')) / 100

        mask1 = yfit_train > t
        X1, Y1 = X_train[mask1], Y_train[mask1]
        mod1 = quantile_regression(X1, Y1, q_val)
        rain1 = reconstruct_rain(X1, mod1)

        mask2 = rain1 > 0
        X2, Y2 = X1[mask2], Y1[mask2]
        mod2 = quantile_regression(X2, Y2, q_val)
        coefs2[label] = mod2
        rain_test = reconstruct_rain(X_test, mod2)
        rain_final[label] = np.maximum(rain_test, 0)

    return coefs2, rain_final

def extract_jjas(series):
    series.index = pd.to_datetime(series.index)
    return series[series.index.month.isin([6,7,8,9])]

def compute_cqvss(y_true, y_pred_model, y_pred_ref, q):
    errors_m = y_true - y_pred_model
    errors_r = y_true - y_pred_ref
    ql_m = np.mean(np.maximum(q * errors_m, (q - 1) * errors_m))
    ql_r = np.mean(np.maximum(q * errors_r, (q - 1) * errors_r))
    return 1 - (ql_m / ql_r) if ql_r != 0 else np.nan

# === CONFIG ===

pca_folder = r'D:/D/Ruvision/ERA5/Pickle format/PCA/Logistic_reg'
obs_excel = r'C:/Users/Angshudeep Majumdar/Documents/Angshudeep Lappy/Ruvision/IMD_2015-2024_Daily Data_0.25 resolution Rain.xlsx'
output_base = r'D:/D/Ruvision/plots/ERA/Logistic_reg'

thresholds = {'80p': 0.2, '85p': 0.15, '90p': 0.1, '95p': 0.05, '99p': 0.01}
quantiles = [0.8, 0.85, 0.9, 0.95, 0.99]
all_conf_matrices=[]

# === MAIN LOOP ===

for pkl in glob.glob(os.path.join(pca_folder, '*.pkl')):
    var_comb = os.path.basename(pkl).split('transformed_dataX_')[1].split('.pkl')[0]
    out_dir = os.path.join(output_base, var_comb)
    os.makedirs(out_dir, exist_ok=True)

    # Load & split
    X, Y = load_data(pkl, obs_excel)
    X_train, X_test, Y_train, Y_test = split_data(X, Y)

    # Run modular 2-step CQM
    coefs2, rain_final = cqm_pipeline(X_train, Y_train, X_test, thresholds)

    # Run SQM
    sqm_models = []
    sqm_final = {}
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q)
        sqm_models.append(mod)
        sqm_final[f"{int(q*100)}p"] = np.maximum(mod.predict(sm.add_constant(X_test)), 0)

    # JJAS
    Y_test.index = pd.to_datetime(Y_test.index)
    mask_jjas = Y_test.index.month.isin([6,7,8,9])
    Y_jjas = Y_test[mask_jjas]
    X_test_jjas = X_test[mask_jjas]

    rain_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in rain_final.items()}
    sqm_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in sqm_final.items()}

    # Linear Regression
    mask_train_jjas = Y_train.index.month.isin([6,7,8,9])
    lr = LinearRegression().fit(X_train[mask_train_jjas], Y_train[mask_train_jjas])
    lr_pred = lr.predict(X_test_jjas)

    # === CONTINGENCY + METRICS ===

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas)
    y_pred_ref = lr_pred

    results = []
    reports=[]
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final_jjas: continue

        y_pred_cqm = rain_final_jjas[q_label].values
        y_pred_sqm = sqm_final_jjas[q_label].values

    # === CORRECT SQM ===
    sqm_models = []
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q, max_iter=5000)
        sqm_models.append(mod)

    sqm_final = {}
    for i, q in enumerate(quantiles):
        rain_q = sqm_models[i].predict(sm.add_constant(X_test))
        rain_q = np.maximum(rain_q, 0)
        sqm_final[f"{int(q*100)}p"] = pd.Series(rain_q, index=X_test.index)

    # === Filter SQM for JJAS ===
    sqm_final_jjas = {}
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label in sqm_final:
            sqm_final_jjas[q_label] = sqm_final[q_label][sqm_final[q_label].index.month.isin([6,7,8,9])]

    # === Train/Test JJAS split ===
    train_index = Y_train.index
    mask_train_jjas = train_index.month.isin([6, 7, 8, 9])
    Y_train_jjas = Y_train[mask_train_jjas]
    X_train_jjas = X_train[mask_train_jjas]
    test_index = Y_test.index
    mask_test_jjas = test_index.month.isin([6, 7, 8, 9])
    X_test_jjas = X_test[mask_test_jjas]
    Y_jjas = Y_test[mask_test_jjas]

    # === Linear Regression ===
    lr = LinearRegression().fit(X_train_jjas, Y_train_jjas)
    pred_lr_jjas = lr.predict(X_test_jjas)
    pred_lr_jjas = pd.Series(pred_lr_jjas, index=X_test_jjas.index)

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas).flatten()
    y_pred_ref = np.array(pred_lr_jjas).flatten()

    results = []

    for q in quantiles:
        q_label = f"{int(q*100)}p"

        if q_label not in rain_final or q_label not in sqm_final_jjas:
            continue

        y_pred_cqm = np.array(pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]).flatten()
        y_pred_sqm = np.array(sqm_final_jjas[q_label]).flatten()

        obs_bin = y_true > threshold_value
        pred_bin_cqm = y_pred_cqm > threshold_value
        pred_bin_sqm = y_pred_sqm > threshold_value
        pred_bin_lr = y_pred_ref > threshold_value

        H = np.sum((obs_bin == 1) & (pred_bin_cqm == 1))
        M = np.sum((obs_bin == 1) & (pred_bin_cqm == 0))
        FA = np.sum((obs_bin == 0) & (pred_bin_cqm == 1))
        CN = np.sum((obs_bin == 0) & (pred_bin_cqm == 0))

        POD = H / (H + M) if (H + M) > 0 else np.nan
        FAR = FA / (H + FA) if (H + FA) > 0 else np.nan
        Accuracy = (H + CN) / (H + M + FA + CN) if (H + M + FA + CN) > 0 else np.nan
        Precision = H / (H + FA) if (H + FA) > 0 else np.nan
        F1 = 2 * (Precision * POD) / (Precision + POD) if (Precision + POD) > 0 else np.nan
        TS = H / (H + M + FA) if (H + M + FA) > 0 else np.nan
        BIAS = (H + FA) / (H + M) if (H + M) > 0 else np.nan
        denom = ((H + M) * (M + CN)) + ((H + FA) * (FA + CN))
        HSS = (2 * (H * CN - FA * M)) / denom if denom != 0 else np.nan
        corr, _ = pearsonr(y_true, y_pred_cqm)
        cqvss = compute_cqvss(y_true, y_pred_cqm, y_pred_ref, q)

        H_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 1))
        M_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 0))
        FA_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 1))
        CN_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 0))
        POD_LR = H_LR / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        FAR_LR = FA_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        Accuracy_LR = (H_LR + CN_LR) / (H_LR + M_LR + FA_LR + CN_LR) if (H_LR + M_LR + FA_LR + CN_LR) > 0 else np.nan
        Precision_LR = H_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        F1_LR = 2 * (Precision_LR * H_LR) / (Precision_LR + H_LR) if (Precision_LR + H_LR) > 0 else np.nan
        TS_LR = H_LR / (H_LR + M_LR + FA_LR) if (H_LR + M_LR + FA_LR) > 0 else np.nan
        BIAS_LR = (H_LR + FA_LR) / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        corr_LR, _ = pearsonr(y_true, y_pred_ref)
        
        H_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 1))
        M_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 0))
        FA_SQM = np.sum((obs_bin == 0) & (pred_bin_sqm == 1))
        CN_SQM= np.sum((obs_bin == 0) & (pred_bin_sqm == 0))
        POD_SQM = H_SQM / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        FAR_SQM = FA_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        Accuracy_SQM = (H_SQM + CN_SQM) / (H_SQM + M_SQM + FA_SQM + CN_SQM) if (H_SQM + M_SQM + FA_SQM + CN_SQM) > 0 else np.nan
        Precision_SQM = H_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        F1_SQM = 2 * (Precision_SQM * H_SQM) / (Precision_SQM + H_SQM) if (Precision_SQM + H_SQM) > 0 else np.nan
        TS_SQM = H_SQM / (H_SQM + M_SQM + FA_SQM) if (H_SQM + M_SQM + FA_SQM) > 0 else np.nan
        BIAS_SQM = (H_SQM + FA_SQM) / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        denom_SQM = ((H_SQM + M_SQM) * (M_SQM + CN_SQM)) + ((H_SQM + FA_SQM) * (FA_SQM + CN_SQM))
        HSS_SQM = (2 * (H_SQM * CN_SQM - FA_SQM * M_SQM)) / denom_SQM if denom_SQM != 0 else np.nan
        corr_sqm, _ = pearsonr(y_true, y_pred_sqm)
        cqvss_sqm = compute_cqvss(y_true, y_pred_sqm, y_pred_ref, q)

        results.append({
            'Quantile': q,
            'Hits_CQM': H, 'Misses_CQM': M, 'False Alarms_CQM': FA, 'Correct Negatives_CQM': CN,
            'Correlation_CQM': round(corr, 3),
            'Hit Rate_CQM': round(POD, 3),
            'False Alarm Ratio_CQM': round(FAR, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'Precision_CQM': round(Precision, 3),
            'F1 Score_CQM': round(F1, 3),
            'Threat Score_CQM': round(TS, 3),
            'Bias Score_CQM': round(BIAS, 3),
            'Heidke Skill Score_CQM': round(HSS, 3),
            'CQVSS_CQM': round(cqvss, 3),
            'Hit Rate_LR': round(POD_LR, 3),
            'False Alarm Ratio_LR': round(FAR_LR, 3),
            'Accuracy_LR': round(Accuracy_LR, 3),
            'Precision_LR': round(Precision_LR, 3),
            'F1 Score_LR': round(F1_LR, 3),
            'Threat Score_LR': round(TS_LR, 3),
            'Bias_LR': round(BIAS_LR, 3),
            'Correlation_LR':round(corr_LR,3),
            'Hits_SQM': H_SQM, 'Misses_SQM': M_SQM, 'False Alarms_SQM': FA_SQM,
            'Correlation_SQM': round(corr_sqm, 3),
            'Hit Rate_SQM': round(POD_SQM, 3),
            'False Alarm Ratio_SQM': round(FAR_SQM, 3),
            'Accuracy_SQM': round(Accuracy_SQM, 3),
            'Precision_SQM': round(Precision_SQM, 3),
            'F1 Score_SQM': round(F1_SQM, 3),
            'Threat Score_SQM': round(TS_SQM, 3),
            'Bias_SQM': round(BIAS_SQM, 3),
            'Heidke Skill Score_SQM': round(HSS_SQM, 3),
            'SQVSS_SQM': round(cqvss_sqm, 3)
        })
        reports.append({
            'Quantile': q,
            'Hits_CQM': H,
            'Misses_CQM': M,
            'False Alarms_CQM': FA,
            'Correct Negatives_CQM': CN,
            'Recall_CQM': round(POD, 3),
            'Precision_CQM': round(Precision, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'F1 Score_CQM': round(F1, 3)
        })
    
    pd.DataFrame(results).to_excel(os.path.join(out_dir, 'contingency_matrix.xlsx'), index=False)

    conf_matrix = pd.DataFrame([
        {
            'Variable_Combination': var_comb,
            **r
        } for r in reports
    ])
    all_conf_matrices.append(conf_matrix)

    # === SAVE FINAL CONCATENATED CONFUSION MATRIX ===

    if all_conf_matrices:
        final_df = pd.concat(all_conf_matrices, ignore_index=True)
        final_df.to_excel(r'C:\Users\Angshudeep Majumdar\Downloads\all_confusion_matrices.xlsx', index=False)


    # === Plots stay same, but now use sqm_final_jjas ===

    # === CQM Plots ===
    fig, axs = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final: continue
        cqm_jjas = pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]
        r, _ = pearsonr(Y_jjas, cqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, cqm_jjas))
        axs[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs[i].plot(cqm_jjas.index, cqm_jjas, label=f'CQM Q{q}', color='red')
        axs[i].set_title(f'CQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'LGR based CQR Model Experiment | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(os.path.join(out_dir, f'QuantilePlots_{var_comb}_LGR based CQR Model Experiment.png'))
    plt.close(fig)

    # === SQM Plots ===
    fig2, axs2 = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in sqm_final_jjas: continue
        sqm_jjas = sqm_final_jjas[q_label]
        r, _ = pearsonr(Y_jjas, sqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, sqm_jjas))
        axs2[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs2[i].plot(sqm_jjas.index, sqm_jjas, label=f'SQM Q{q}', color='green')
        axs2[i].set_title(f'SQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs2[i].tick_params(axis='x', rotation=45)
        axs2[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'SQM Prediction | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig2.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig2.tight_layout(rect=[0, 0, 1, 0.95])
    fig2.savefig(os.path.join(out_dir, f'SQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig2)

    # === Skill Score Plots ===
    metrics_df = pd.DataFrame(results)
    x_labels = [f"{int(q)}p" for q in metrics_df['Quantile'] * 100]

    fig3, axs3 = plt.subplots(1, 6, figsize=(30, 6))
    axs3[0].bar(x_labels, metrics_df['Hit Rate_CQM'], color='red'); axs3[0].set_title('Hit Rate (POD)')
    axs3[1].bar(x_labels, metrics_df['False Alarm Ratio_CQM'], color='orange'); axs3[1].set_title('False Alarm Ratio (FAR)')
    axs3[2].bar(x_labels, metrics_df['Threat Score_CQM'], color='blue'); axs3[2].set_title('Threat Score (TS)')
    axs3[3].bar(x_labels, metrics_df['Bias Score_CQM'], color='purple'); axs3[3].set_title('Bias Score')
    axs3[4].bar(x_labels, metrics_df['Heidke Skill Score_CQM'], color='green'); axs3[4].set_title('HSS')
    axs3[5].bar(x_labels, metrics_df['CQVSS_CQM'], color='pink'); axs3[5].set_title('CQVSS')
    fig3.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_LGR based CQR Model Experiment', fontsize=16)
    fig3.tight_layout(rect=[0, 0, 1, 0.95])
    fig3.savefig(os.path.join(out_dir, f'CQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig3)

    fig4, axs4 = plt.subplots(1, 6, figsize=(30, 6))
    axs4[0].bar(x_labels, metrics_df['Hit Rate_SQM'], color='red'); axs4[0].set_title('Hit Rate (POD)')
    axs4[1].bar(x_labels, metrics_df['False Alarm Ratio_SQM'], color='orange'); axs4[1].set_title('False Alarm Ratio (FAR)')
    axs4[2].bar(x_labels, metrics_df['Threat Score_SQM'], color='blue'); axs4[2].set_title('Threat Score (TS)')
    axs4[3].bar(x_labels, metrics_df['Bias_SQM'], color='purple'); axs4[3].set_title('Bias Score')
    axs4[4].bar(x_labels, metrics_df['Heidke Skill Score_SQM'], color='green'); axs4[4].set_title('HSS')
    axs4[5].bar(x_labels, metrics_df['SQVSS_SQM'], color='pink'); axs4[5].set_title('SQVSS')
    fig4.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_SQM', fontsize=16)
    fig4.tight_layout(rect=[0, 0, 1, 0.95])
    fig4.savefig(os.path.join(out_dir, f'SQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig4)

    # === Linear Regression ===
    fig5, ax5 = plt.subplots(figsize=(15, 6))
    r_lr, _ = pearsonr(Y_jjas, pred_lr_jjas)
    rmse_lr = np.sqrt(mean_squared_error(Y_jjas, pred_lr_jjas))
    ax5.plot(Y_jjas.index, Y_jjas, label='Observed', color='blue')
    ax5.plot(pred_lr_jjas.index, pred_lr_jjas, label='Linear Regression', color='black')
    ax5.set_title(f'Linear Regression | {var_comb} | JJAS | r={r_lr:.2f} | RMSE={rmse_lr:.2f}', fontsize=14)
    ax5.tick_params(axis='x', rotation=45)
    ax5.legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'Linear Regression | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig5.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig5.tight_layout(rect=[0, 0, 1, 0.95])
    fig5.savefig(os.path.join(out_dir, f'LinearRegression_{var_comb}_JJAS.png'))
    plt.close(fig5)


C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar

In [7]:
# Using Random Forest to calculate Y_fit_train and Y_fit_test
import os
import glob
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# === FUNCTIONS ===

def load_data(pkl_path, excel_path):
    X = pd.read_pickle(pkl_path)
    Y = pd.read_excel(excel_path).set_index('DateTime')['Prec_23.0_72.5']
    return X, Y

def split_data(X, Y, split_index=3287):
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    Y_train, Y_test = Y.iloc[:split_index], Y.iloc[split_index:]
    return X_train, X_test, Y_train, Y_test


def binary_rf(X_train, Y_train):
    yb_train = (Y_train > 0).astype(int)
    rf = RandomForestClassifier(n_estimators=50, random_state=30)
    rf.fit(X_train, yb_train)
    yfit_train = rf.predict_proba(X_train)[:, 1]  # probability P(Y>0)
    return yfit_train


def quantile_regression(X, Y, q):
    mod = sm.QuantReg(Y, sm.add_constant(X)).fit(q=q)
    return mod.params

def reconstruct_rain(X, coefs):
    return sm.add_constant(X) @ coefs

def cqm_pipeline(X_train, Y_train, X_test, thresholds):
    yfit_train = binary_rf(X_train, Y_train)

    coefs2 = {}
    rain_final = {}

    for label, t in thresholds.items():
        q_val = float(label.replace('p','')) / 100

        mask1 = yfit_train > t
        X1, Y1 = X_train[mask1], Y_train[mask1]
        mod1 = quantile_regression(X1, Y1, q_val)
        rain1 = reconstruct_rain(X1, mod1)

        mask2 = rain1 > 0
        X2, Y2 = X1[mask2], Y1[mask2]
        mod2 = quantile_regression(X2, Y2, q_val)
        coefs2[label] = mod2
        rain_test = reconstruct_rain(X_test, mod2)
        rain_final[label] = np.maximum(rain_test, 0)

    return coefs2, rain_final

def extract_jjas(series):
    series.index = pd.to_datetime(series.index)
    return series[series.index.month.isin([6,7,8,9])]

def compute_cqvss(y_true, y_pred_model, y_pred_ref, q):
    errors_m = y_true - y_pred_model
    errors_r = y_true - y_pred_ref
    ql_m = np.mean(np.maximum(q * errors_m, (q - 1) * errors_m))
    ql_r = np.mean(np.maximum(q * errors_r, (q - 1) * errors_r))
    return 1 - (ql_m / ql_r) if ql_r != 0 else np.nan

# === CONFIG ===

pca_folder = r'D:/D/Ruvision/ERA5/Pickle format/PCA/Logistic_reg'
obs_excel = r'C:/Users/Angshudeep Majumdar/Documents/Angshudeep Lappy/Ruvision/IMD_2015-2024_Daily Data_0.25 resolution Rain.xlsx'
output_base = r'D:/D/Ruvision/plots/ERA/Random_Forest'

thresholds = {'80p': 0.2, '85p': 0.15, '90p': 0.1, '95p': 0.05, '99p': 0.01}
quantiles = [0.8, 0.85, 0.9, 0.95, 0.99]
all_conf_matrices=[]
# === MAIN LOOP ===

for pkl in glob.glob(os.path.join(pca_folder, '*.pkl')):
    var_comb = os.path.basename(pkl).split('transformed_dataX_')[1].split('.pkl')[0]
    out_dir = os.path.join(output_base, var_comb)
    os.makedirs(out_dir, exist_ok=True)

    # Load & split
    X, Y = load_data(pkl, obs_excel)
    X_train, X_test, Y_train, Y_test = split_data(X, Y)

    # Run modular 2-step CQM
    coefs2, rain_final = cqm_pipeline(X_train, Y_train, X_test, thresholds)

    # Run SQM
    sqm_models = []
    sqm_final = {}
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q)
        sqm_models.append(mod)
        sqm_final[f"{int(q*100)}p"] = np.maximum(mod.predict(sm.add_constant(X_test)), 0)

    # JJAS
    Y_test.index = pd.to_datetime(Y_test.index)
    mask_jjas = Y_test.index.month.isin([6,7,8,9])
    Y_jjas = Y_test[mask_jjas]
    X_test_jjas = X_test[mask_jjas]

    rain_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in rain_final.items()}
    sqm_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in sqm_final.items()}

    # Linear Regression
    mask_train_jjas = Y_train.index.month.isin([6,7,8,9])
    lr = LinearRegression().fit(X_train[mask_train_jjas], Y_train[mask_train_jjas])
    lr_pred = lr.predict(X_test_jjas)

    # === CONTINGENCY + METRICS ===

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas)
    y_pred_ref = lr_pred

    results = []
    reports=[]
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final_jjas: continue

        y_pred_cqm = rain_final_jjas[q_label].values
        y_pred_sqm = sqm_final_jjas[q_label].values

    # === CORRECT SQM ===
    sqm_models = []
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q, max_iter=5000)
        sqm_models.append(mod)

    sqm_final = {}
    for i, q in enumerate(quantiles):
        rain_q = sqm_models[i].predict(sm.add_constant(X_test))
        rain_q = np.maximum(rain_q, 0)
        sqm_final[f"{int(q*100)}p"] = pd.Series(rain_q, index=X_test.index)

    # === Filter SQM for JJAS ===
    sqm_final_jjas = {}
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label in sqm_final:
            sqm_final_jjas[q_label] = sqm_final[q_label][sqm_final[q_label].index.month.isin([6,7,8,9])]

    # === Train/Test JJAS split ===
    train_index = Y_train.index
    mask_train_jjas = train_index.month.isin([6, 7, 8, 9])
    Y_train_jjas = Y_train[mask_train_jjas]
    X_train_jjas = X_train[mask_train_jjas]
    test_index = Y_test.index
    mask_test_jjas = test_index.month.isin([6, 7, 8, 9])
    X_test_jjas = X_test[mask_test_jjas]
    Y_jjas = Y_test[mask_test_jjas]

    # === Linear Regression ===
    lr = LinearRegression().fit(X_train_jjas, Y_train_jjas)
    pred_lr_jjas = lr.predict(X_test_jjas)
    pred_lr_jjas = pd.Series(pred_lr_jjas, index=X_test_jjas.index)

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas).flatten()
    y_pred_ref = np.array(pred_lr_jjas).flatten()

    results = []
    reports=[]

    for q in quantiles:
        q_label = f"{int(q * 100)}p"
        if q_label not in rain_final_jjas or q_label not in sqm_final_jjas:
            continue

        y_pred_cqm = rain_final_jjas[q_label].values
        y_pred_sqm = sqm_final_jjas[q_label].values
        obs_bin = y_true > threshold_value
        pred_bin_cqm = y_pred_cqm > threshold_value
        pred_bin_sqm = y_pred_sqm > threshold_value
        pred_bin_lr = y_pred_ref > threshold_value

        H = np.sum((obs_bin == 1) & (pred_bin_cqm == 1))
        M = np.sum((obs_bin == 1) & (pred_bin_cqm == 0))
        FA = np.sum((obs_bin == 0) & (pred_bin_cqm == 1))
        CN = np.sum((obs_bin == 0) & (pred_bin_cqm == 0))

        POD = H / (H + M) if (H + M) > 0 else np.nan
        FAR = FA / (H + FA) if (H + FA) > 0 else np.nan
        Accuracy = (H + CN) / (H + M + FA + CN) if (H + M + FA + CN) > 0 else np.nan
        Precision = H / (H + FA) if (H + FA) > 0 else np.nan
        F1 = 2 * (Precision * POD) / (Precision + POD) if (Precision + POD) > 0 else np.nan
        TS = H / (H + M + FA) if (H + M + FA) > 0 else np.nan
        BIAS = (H + FA) / (H + M) if (H + M) > 0 else np.nan
        denom = ((H + M) * (M + CN)) + ((H + FA) * (FA + CN))
        HSS = (2 * (H * CN - FA * M)) / denom if denom != 0 else np.nan
        corr, _ = pearsonr(y_true, y_pred_cqm)
        cqvss = compute_cqvss(y_true, y_pred_cqm, y_pred_ref, q)

        H_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 1))
        M_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 0))
        FA_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 1))
        CN_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 0))
        POD_LR = H_LR / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        FAR_LR = FA_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        Accuracy_LR = (H_LR + CN_LR) / (H_LR + M_LR + FA_LR + CN_LR) if (H_LR + M_LR + FA_LR + CN_LR) > 0 else np.nan
        Precision_LR = H_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        F1_LR = 2 * (Precision_LR * H_LR) / (Precision_LR + H_LR) if (Precision_LR + H_LR) > 0 else np.nan
        TS_LR = H_LR / (H_LR + M_LR + FA_LR) if (H_LR + M_LR + FA_LR) > 0 else np.nan
        BIAS_LR = (H_LR + FA_LR) / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        corr_LR, _ = pearsonr(y_true, y_pred_ref)
        
        H_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 1))
        M_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 0))
        FA_SQM = np.sum((obs_bin == 0) & (pred_bin_sqm == 1))
        CN_SQM= np.sum((obs_bin == 0) & (pred_bin_sqm == 0))
        POD_SQM = H_SQM / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        FAR_SQM = FA_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        Accuracy_SQM = (H_SQM + CN_SQM) / (H_SQM + M_SQM + FA_SQM + CN_SQM) if (H_SQM + M_SQM + FA_SQM + CN_SQM) > 0 else np.nan
        Precision_SQM = H_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        F1_SQM = 2 * (Precision_SQM * H_SQM) / (Precision_SQM + H_SQM) if (Precision_SQM + H_SQM) > 0 else np.nan
        TS_SQM = H_SQM / (H_SQM + M_SQM + FA_SQM) if (H_SQM + M_SQM + FA_SQM) > 0 else np.nan
        BIAS_SQM = (H_SQM + FA_SQM) / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        denom_SQM = ((H_SQM + M_SQM) * (M_SQM + CN_SQM)) + ((H_SQM + FA_SQM) * (FA_SQM + CN_SQM))
        HSS_SQM = (2 * (H_SQM * CN_SQM - FA_SQM * M_SQM)) / denom_SQM if denom_SQM != 0 else np.nan
        corr_sqm, _ = pearsonr(y_true, y_pred_sqm)
        cqvss_sqm = compute_cqvss(y_true, y_pred_sqm, y_pred_ref, q)

        results.append({
            'Quantile': q,
            'Hits_CQM': H, 'Misses_CQM': M, 'False Alarms_CQM': FA, 'Correct Negatives_CQM': CN,
            'Correlation_CQM': round(corr, 3),
            'Hit Rate_CQM': round(POD, 3),
            'False Alarm Ratio_CQM': round(FAR, 3),
            'Precision_CQM': round(Precision, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'Precision_CQM': round(Precision, 3),
            'F1 Score_CQM': round(F1, 3),
            'Threat Score_CQM': round(TS, 3),
            'Bias Score_CQM': round(BIAS, 3),
            'Heidke Skill Score_CQM': round(HSS, 3),
            'CQVSS_CQM': round(cqvss, 3),
            'Hit Rate_LR': round(POD_LR, 3),
            'False Alarm Ratio_LR': round(FAR_LR, 3),
            'Accuracy_LR': round(Accuracy_LR, 3),
            'Precision_LR': round(Precision_LR, 3),
            'F1 Score_LR': round(F1_LR, 3),
            'Threat Score_LR': round(TS_LR, 3),
            'Bias_LR': round(BIAS_LR, 3),
            'Correlation_LR':round(corr_LR,3),
            'Hits_SQM': H_SQM, 'Misses_SQM': M_SQM, 'False Alarms_SQM': FA_SQM,
            'Correlation_SQM': round(corr_sqm, 3),
            'Hit Rate_SQM': round(POD_SQM, 3),
            'False Alarm Ratio_SQM': round(FAR_SQM, 3),
            'Accuracy_SQM': round(Accuracy_SQM, 3),
            'Precision_SQM': round(Precision_SQM, 3),
            'F1 Score_SQM': round(F1_SQM, 3),
            'Threat Score_SQM': round(TS_SQM, 3),
            'Bias_SQM': round(BIAS_SQM, 3),
            'Heidke Skill Score_SQM': round(HSS_SQM, 3),
            'SQVSS_SQM': round(cqvss_sqm, 3)
        })
        reports.append({
            'Quantile': q,
            'Hits_CQM': H,
            'Misses_CQM': M,
            'False Alarms_CQM': FA,
            'Correct Negatives_CQM': CN,
            'Recall_CQM': round(POD, 3),
            'Precision_CQM': round(Precision, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'F1 Score_CQM': round(F1, 3)
        })
    
    pd.DataFrame(results).to_excel(os.path.join(out_dir, 'contingency_matrix.xlsx'), index=False)

    conf_matrix = pd.DataFrame([
        {
            'Variable_Combination': var_comb,
            **r
        } for r in reports
    ])
    all_conf_matrices.append(conf_matrix)

    # === SAVE FINAL CONCATENATED CONFUSION MATRIX ===

    if all_conf_matrices:
        final_df = pd.concat(all_conf_matrices, ignore_index=True)
        final_df.to_excel(r'C:\Users\Angshudeep Majumdar\Downloads\all_confusion_matrices.xlsx', index=False)

    # === Plots stay same, but now use sqm_final_jjas ===

    # === CQM Plots ===
    fig, axs = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final: continue
        cqm_jjas = pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]
        r, _ = pearsonr(Y_jjas, cqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, cqm_jjas))
        axs[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs[i].plot(cqm_jjas.index, cqm_jjas, label=f'CQM Q{q}', color='red')
        axs[i].set_title(f'CQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'RF based CQR Model Experiment | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(os.path.join(out_dir, f'CQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig)

    # === SQM Plots ===
    fig2, axs2 = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in sqm_final_jjas: continue
        sqm_jjas = sqm_final_jjas[q_label]
        r, _ = pearsonr(Y_jjas, sqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, sqm_jjas))
        axs2[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs2[i].plot(sqm_jjas.index, sqm_jjas, label=f'SQM Q{q}', color='green')
        axs2[i].set_title(f'SQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs2[i].tick_params(axis='x', rotation=45)
        axs2[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'SQM Prediction | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig2.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig2.tight_layout(rect=[0, 0, 1, 0.95])
    fig2.savefig(os.path.join(out_dir, f'SQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig2)

    # === Skill Score Plots ===
    metrics_df = pd.DataFrame(results)
    x_labels = [f"{int(q)}p" for q in metrics_df['Quantile'] * 100]

    fig3, axs3 = plt.subplots(1, 6, figsize=(30, 6))
    axs3[0].bar(x_labels, metrics_df['Hit Rate_CQM'], color='red'); axs3[0].set_title('Hit Rate (POD)')
    axs3[1].bar(x_labels, metrics_df['False Alarm Ratio_CQM'], color='orange'); axs3[1].set_title('False Alarm Ratio (FAR)')
    axs3[2].bar(x_labels, metrics_df['Threat Score_CQM'], color='blue'); axs3[2].set_title('Threat Score (TS)')
    axs3[3].bar(x_labels, metrics_df['Bias Score_CQM'], color='purple'); axs3[3].set_title('Bias Score')
    axs3[4].bar(x_labels, metrics_df['Heidke Skill Score_CQM'], color='green'); axs3[4].set_title('HSS')
    axs3[5].bar(x_labels, metrics_df['CQVSS_CQM'], color='pink'); axs3[5].set_title('CQVSS')
    fig3.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_RF based CQR Model Experiment', fontsize=16)
    fig3.tight_layout(rect=[0, 0, 1, 0.95])
    fig3.savefig(os.path.join(out_dir, f'CQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig3)

    fig4, axs4 = plt.subplots(1, 6, figsize=(30, 6))
    axs4[0].bar(x_labels, metrics_df['Hit Rate_SQM'], color='red'); axs4[0].set_title('Hit Rate (POD)')
    axs4[1].bar(x_labels, metrics_df['False Alarm Ratio_SQM'], color='orange'); axs4[1].set_title('False Alarm Ratio (FAR)')
    axs4[2].bar(x_labels, metrics_df['Threat Score_SQM'], color='blue'); axs4[2].set_title('Threat Score (TS)')
    axs4[3].bar(x_labels, metrics_df['Bias_SQM'], color='purple'); axs4[3].set_title('Bias Score')
    axs4[4].bar(x_labels, metrics_df['Heidke Skill Score_SQM'], color='green'); axs4[4].set_title('HSS')
    axs4[5].bar(x_labels, metrics_df['SQVSS_SQM'], color='pink'); axs4[5].set_title('SQVSS')
    fig4.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_SQM', fontsize=16)
    fig4.tight_layout(rect=[0, 0, 1, 0.95])
    fig4.savefig(os.path.join(out_dir, f'SQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig4)

    # === Linear Regression ===
    fig5, ax5 = plt.subplots(figsize=(15, 6))
    r_lr, _ = pearsonr(Y_jjas, pred_lr_jjas)
    rmse_lr = np.sqrt(mean_squared_error(Y_jjas, pred_lr_jjas))
    ax5.plot(Y_jjas.index, Y_jjas, label='Observed', color='blue')
    ax5.plot(pred_lr_jjas.index, pred_lr_jjas, label='Linear Regression', color='black')
    ax5.set_title(f'Linear Regression | {var_comb} | JJAS | r={r_lr:.2f} | RMSE={rmse_lr:.2f}', fontsize=14)
    ax5.tick_params(axis='x', rotation=45)
    ax5.legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'Linear Regression | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig5.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig5.tight_layout(rect=[0, 0, 1, 0.95])
    fig5.savefig(os.path.join(out_dir, f'LinearRegression_{var_comb}_JJAS.png'))
    plt.close(fig5)


C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar

In [8]:
# Using XGBoost to calculate Y_fit_train and Y_fit_test
import os
import glob
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

# === FUNCTIONS ===

def load_data(pkl_path, excel_path):
    X = pd.read_pickle(pkl_path)
    Y = pd.read_excel(excel_path).set_index('DateTime')['Prec_23.0_72.5']
    return X, Y

def split_data(X, Y, split_index=3287):
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    Y_train, Y_test = Y.iloc[:split_index], Y.iloc[split_index:]
    return X_train, X_test, Y_train, Y_test

def binary_xgb(X_train, Y_train):
    yb_train = (Y_train > 0).astype(int)
    xgb = XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    xgb.fit(X_train, yb_train)
    yfit_train = xgb.predict_proba(X_train)[:, 1]
    return yfit_train
    
def quantile_regression(X, Y, q):
    mod = sm.QuantReg(Y, sm.add_constant(X)).fit(q=q)
    return mod.params

def reconstruct_rain(X, coefs):
    return sm.add_constant(X) @ coefs

def cqm_pipeline(X_train, Y_train, X_test, thresholds):
    yfit_train = binary_xgb(X_train, Y_train)

    coefs2 = {}
    rain_final = {}

    for label, t in thresholds.items():
        q_val = float(label.replace('p','')) / 100

        mask1 = yfit_train > t
        X1, Y1 = X_train[mask1], Y_train[mask1]
        mod1 = quantile_regression(X1, Y1, q_val)
        rain1 = reconstruct_rain(X1, mod1)

        mask2 = rain1 > 0
        X2, Y2 = X1[mask2], Y1[mask2]
        mod2 = quantile_regression(X2, Y2, q_val)
        coefs2[label] = mod2
        rain_test = reconstruct_rain(X_test, mod2)
        rain_final[label] = np.maximum(rain_test, 0)

    return coefs2, rain_final

def extract_jjas(series):
    series.index = pd.to_datetime(series.index)
    return series[series.index.month.isin([6,7,8,9])]

def compute_cqvss(y_true, y_pred_model, y_pred_ref, q):
    errors_m = y_true - y_pred_model
    errors_r = y_true - y_pred_ref
    ql_m = np.mean(np.maximum(q * errors_m, (q - 1) * errors_m))
    ql_r = np.mean(np.maximum(q * errors_r, (q - 1) * errors_r))
    return 1 - (ql_m / ql_r) if ql_r != 0 else np.nan

# === CONFIG ===

pca_folder = r'D:/D/Ruvision/ERA5/Pickle format/PCA/Logistic_reg'
obs_excel = r'C:/Users/Angshudeep Majumdar/Documents/Angshudeep Lappy/Ruvision/IMD_2015-2024_Daily Data_0.25 resolution Rain.xlsx'
output_base = r'D:/D/Ruvision/plots/ERA/XGBoost'

thresholds = {'80p': 0.2, '85p': 0.15, '90p': 0.1, '95p': 0.05, '99p': 0.01}
quantiles = [0.8, 0.85, 0.9, 0.95, 0.99]
all_conf_matrices=[]

# === MAIN LOOP ===

for pkl in glob.glob(os.path.join(pca_folder, '*.pkl')):
    var_comb = os.path.basename(pkl).split('transformed_dataX_')[1].split('.pkl')[0]
    out_dir = os.path.join(output_base, var_comb)
    os.makedirs(out_dir, exist_ok=True)

    # Load & split
    X, Y = load_data(pkl, obs_excel)
    X_train, X_test, Y_train, Y_test = split_data(X, Y)

    # Run modular 2-step CQM
    coefs2, rain_final = cqm_pipeline(X_train, Y_train, X_test, thresholds)

    # Run SQM
    sqm_models = []
    sqm_final = {}
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q)
        sqm_models.append(mod)
        sqm_final[f"{int(q*100)}p"] = np.maximum(mod.predict(sm.add_constant(X_test)), 0)

    # JJAS
    Y_test.index = pd.to_datetime(Y_test.index)
    mask_jjas = Y_test.index.month.isin([6,7,8,9])
    Y_jjas = Y_test[mask_jjas]
    X_test_jjas = X_test[mask_jjas]

    rain_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in rain_final.items()}
    sqm_final_jjas = {k: pd.Series(v, index=X_test.index)[mask_jjas] for k, v in sqm_final.items()}

    # Linear Regression
    mask_train_jjas = Y_train.index.month.isin([6,7,8,9])
    lr = LinearRegression().fit(X_train[mask_train_jjas], Y_train[mask_train_jjas])
    lr_pred = lr.predict(X_test_jjas)

    # === CONTINGENCY + METRICS ===

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas)
    y_pred_ref = lr_pred

    results = []
    reports=[]
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final_jjas: continue

        y_pred_cqm = rain_final_jjas[q_label].values
        y_pred_sqm = sqm_final_jjas[q_label].values

    # === CORRECT SQM ===
    sqm_models = []
    for q in quantiles:
        mod = sm.QuantReg(Y_train, sm.add_constant(X_train)).fit(q=q, max_iter=5000)
        sqm_models.append(mod)

    sqm_final = {}
    for i, q in enumerate(quantiles):
        rain_q = sqm_models[i].predict(sm.add_constant(X_test))
        rain_q = np.maximum(rain_q, 0)
        sqm_final[f"{int(q*100)}p"] = pd.Series(rain_q, index=X_test.index)

    # === Filter SQM for JJAS ===
    sqm_final_jjas = {}
    for q in quantiles:
        q_label = f"{int(q*100)}p"
        if q_label in sqm_final:
            sqm_final_jjas[q_label] = sqm_final[q_label][sqm_final[q_label].index.month.isin([6,7,8,9])]

    # === Train/Test JJAS split ===
    train_index = Y_train.index
    mask_train_jjas = train_index.month.isin([6, 7, 8, 9])
    Y_train_jjas = Y_train[mask_train_jjas]
    X_train_jjas = X_train[mask_train_jjas]
    test_index = Y_test.index
    mask_test_jjas = test_index.month.isin([6, 7, 8, 9])
    X_test_jjas = X_test[mask_test_jjas]
    Y_jjas = Y_test[mask_test_jjas]

    # === Linear Regression ===
    lr = LinearRegression().fit(X_train_jjas, Y_train_jjas)
    pred_lr_jjas = lr.predict(X_test_jjas)
    pred_lr_jjas = pd.Series(pred_lr_jjas, index=X_test_jjas.index)

    threshold_value = np.percentile(Y_jjas, 95)
    y_true = np.array(Y_jjas).flatten()
    y_pred_ref = np.array(pred_lr_jjas).flatten()

    results = []

    for q in quantiles:
        q_label = f"{int(q*100)}p"

        if q_label not in rain_final or q_label not in sqm_final_jjas:
            continue

        y_pred_cqm = np.array(pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]).flatten()
        y_pred_sqm = np.array(sqm_final_jjas[q_label]).flatten()

        obs_bin = y_true > threshold_value
        pred_bin_cqm = y_pred_cqm > threshold_value
        pred_bin_sqm = y_pred_sqm > threshold_value
        pred_bin_lr = y_pred_ref > threshold_value

        H = np.sum((obs_bin == 1) & (pred_bin_cqm == 1))
        M = np.sum((obs_bin == 1) & (pred_bin_cqm == 0))
        FA = np.sum((obs_bin == 0) & (pred_bin_cqm == 1))
        CN = np.sum((obs_bin == 0) & (pred_bin_cqm == 0))

        POD = H / (H + M) if (H + M) > 0 else np.nan
        FAR = FA / (H + FA) if (H + FA) > 0 else np.nan
        Accuracy = (H + CN) / (H + M + FA + CN) if (H + M + FA + CN) > 0 else np.nan
        Precision = H / (H + FA) if (H + FA) > 0 else np.nan
        F1 = 2 * (Precision * POD) / (Precision + POD) if (Precision + POD) > 0 else np.nan
        TS = H / (H + M + FA) if (H + M + FA) > 0 else np.nan
        BIAS = (H + FA) / (H + M) if (H + M) > 0 else np.nan
        denom = ((H + M) * (M + CN)) + ((H + FA) * (FA + CN))
        HSS = (2 * (H * CN - FA * M)) / denom if denom != 0 else np.nan
        corr, _ = pearsonr(y_true, y_pred_cqm)
        cqvss = compute_cqvss(y_true, y_pred_cqm, y_pred_ref, q)

        H_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 1))
        M_LR = np.sum((obs_bin == 1) & (pred_bin_lr == 0))
        FA_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 1))
        CN_LR = np.sum((obs_bin == 0) & (pred_bin_lr == 0))
        POD_LR = H_LR / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        FAR_LR = FA_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        Accuracy_LR = (H_LR + CN_LR) / (H_LR + M_LR + FA_LR + CN_LR) if (H_LR + M_LR + FA_LR + CN_LR) > 0 else np.nan
        Precision_LR = H_LR / (H_LR + FA_LR) if (H_LR + FA_LR) > 0 else np.nan
        F1_LR = 2 * (Precision_LR * H_LR) / (Precision_LR + H_LR) if (Precision_LR + H_LR) > 0 else np.nan
        TS_LR = H_LR / (H_LR + M_LR + FA_LR) if (H_LR + M_LR + FA_LR) > 0 else np.nan
        BIAS_LR = (H_LR + FA_LR) / (H_LR + M_LR) if (H_LR + M_LR) > 0 else np.nan
        corr_LR, _ = pearsonr(y_true, y_pred_ref)
        
        H_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 1))
        M_SQM = np.sum((obs_bin == 1) & (pred_bin_sqm == 0))
        FA_SQM = np.sum((obs_bin == 0) & (pred_bin_sqm == 1))
        CN_SQM= np.sum((obs_bin == 0) & (pred_bin_sqm == 0))
        POD_SQM = H_SQM / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        FAR_SQM = FA_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        Accuracy_SQM = (H_SQM + CN_SQM) / (H_SQM + M_SQM + FA_SQM + CN_SQM) if (H_SQM + M_SQM + FA_SQM + CN_SQM) > 0 else np.nan
        Precision_SQM = H_SQM / (H_SQM + FA_SQM) if (H_SQM + FA_SQM) > 0 else np.nan
        F1_SQM = 2 * (Precision_SQM * H_SQM) / (Precision_SQM + H_SQM) if (Precision_SQM + H_SQM) > 0 else np.nan
        TS_SQM = H_SQM / (H_SQM + M_SQM + FA_SQM) if (H_SQM + M_SQM + FA_SQM) > 0 else np.nan
        BIAS_SQM = (H_SQM + FA_SQM) / (H_SQM + M_SQM) if (H_SQM + M_SQM) > 0 else np.nan
        denom_SQM = ((H_SQM + M_SQM) * (M_SQM + CN_SQM)) + ((H_SQM + FA_SQM) * (FA_SQM + CN_SQM))
        HSS_SQM = (2 * (H_SQM * CN_SQM - FA_SQM * M_SQM)) / denom_SQM if denom_SQM != 0 else np.nan
        corr_sqm, _ = pearsonr(y_true, y_pred_sqm)
        cqvss_sqm = compute_cqvss(y_true, y_pred_sqm, y_pred_ref, q)

        results.append({
            'Quantile': q,
            'Hits_CQM': H, 'Misses_CQM': M, 'False Alarms_CQM': FA, 'Correct Negatives_CQM': CN,
            'Correlation_CQM': round(corr, 3),
            'Hit Rate_CQM': round(POD, 3),
            'False Alarm Ratio_CQM': round(FAR, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'Precision_CQM': round(Precision, 3),
            'F1 Score_CQM': round(F1, 3),
            'Threat Score_CQM': round(TS, 3),
            'Bias Score_CQM': round(BIAS, 3),
            'Heidke Skill Score_CQM': round(HSS, 3),
            'CQVSS_CQM': round(cqvss, 3),
            'Hit Rate_LR': round(POD_LR, 3),
            'False Alarm Ratio_LR': round(FAR_LR, 3),
            'Accuracy_LR': round(Accuracy_LR, 3),
            'Precision_LR': round(Precision_LR, 3),
            'F1 Score_LR': round(F1_LR, 3),
            'Threat Score_LR': round(TS_LR, 3),
            'Bias_LR': round(BIAS_LR, 3),
            'Correlation_LR':round(corr_LR,3),
            'Hits_SQM': H_SQM, 'Misses_SQM': M_SQM, 'False Alarms_SQM': FA_SQM,
            'Correlation_SQM': round(corr_sqm, 3),
            'Hit Rate_SQM': round(POD_SQM, 3),
            'False Alarm Ratio_SQM': round(FAR_SQM, 3),
            'Accuracy_SQM': round(Accuracy_SQM, 3),
            'Precision_SQM': round(Precision_SQM, 3),
            'F1 Score_SQM': round(F1_SQM, 3),
            'Threat Score_SQM': round(TS_SQM, 3),
            'Bias_SQM': round(BIAS_SQM, 3),
            'Heidke Skill Score_SQM': round(HSS_SQM, 3),
            'SQVSS_SQM': round(cqvss_sqm, 3)
        
        })
        reports.append({
            'Quantile': q,
            'Hits_CQM': H,
            'Misses_CQM': M,
            'False Alarms_CQM': FA,
            'Correct Negatives_CQM': CN,
            'Recall_CQM': round(POD, 3),
            'Precision_CQM': round(Precision, 3),
            'Accuracy_CQM': round(Accuracy, 3),
            'F1 Score_CQM': round(F1, 3)
        })
    
    pd.DataFrame(results).to_excel(os.path.join(out_dir, 'contingency_matrix.xlsx'), index=False)

    conf_matrix = pd.DataFrame([
        {
            'Variable_Combination': var_comb,
            **r
        } for r in reports
    ])
    all_conf_matrices.append(conf_matrix)

    # === SAVE FINAL CONCATENATED CONFUSION MATRIX ===

    if all_conf_matrices:
        final_df = pd.concat(all_conf_matrices, ignore_index=True)
        final_df.to_excel(r'C:\Users\Angshudeep Majumdar\Downloads\all_confusion_matrices.xlsx', index=False)

       
    # === CQM Plots ===
    fig, axs = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in rain_final: continue
        cqm_jjas = pd.Series(rain_final[q_label], index=test_index)[mask_test_jjas]
        r, _ = pearsonr(Y_jjas, cqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, cqm_jjas))
        axs[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs[i].plot(cqm_jjas.index, cqm_jjas, label=f'CQM Q{q}', color='red')
        axs[i].set_title(f'CQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'XGBOOST based CQR Model Experiment | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(os.path.join(out_dir, f'CQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig)

    # === SQM Plots ===
    fig2, axs2 = plt.subplots(1, 5, figsize=(30, 6), sharey=True)
    for i, q in enumerate(quantiles):
        q_label = f"{int(q*100)}p"
        if q_label not in sqm_final_jjas: continue
        sqm_jjas = sqm_final_jjas[q_label]
        r, _ = pearsonr(Y_jjas, sqm_jjas)
        rmse = np.sqrt(mean_squared_error(Y_jjas, sqm_jjas))
        axs2[i].plot(Y_jjas.index, Y_jjas, label='Obs', color='blue')
        axs2[i].plot(sqm_jjas.index, sqm_jjas, label=f'SQM Q{q}', color='green')
        axs2[i].set_title(f'SQM Q{q} | r={r:.2f} | RMSE={rmse:.2f}')
        axs2[i].tick_params(axis='x', rotation=45)
        axs2[i].legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'SQM Prediction | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig2.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig2.tight_layout(rect=[0, 0, 1, 0.95])
    fig2.savefig(os.path.join(out_dir, f'SQM_QuantilePlots_{var_comb}.png'))
    plt.close(fig2)

    # === Skill Score Plots ===
    metrics_df = pd.DataFrame(results)
    x_labels = [f"{int(q)}p" for q in metrics_df['Quantile'] * 100]

    fig3, axs3 = plt.subplots(1, 6, figsize=(30, 6))
    axs3[0].bar(x_labels, metrics_df['Hit Rate_CQM'], color='red'); axs3[0].set_title('Hit Rate (POD)')
    axs3[1].bar(x_labels, metrics_df['False Alarm Ratio_CQM'], color='orange'); axs3[1].set_title('False Alarm Ratio (FAR)')
    axs3[2].bar(x_labels, metrics_df['Threat Score_CQM'], color='blue'); axs3[2].set_title('Threat Score (TS)')
    axs3[3].bar(x_labels, metrics_df['Bias Score_CQM'], color='purple'); axs3[3].set_title('Bias Score')
    axs3[4].bar(x_labels, metrics_df['Heidke Skill Score_CQM'], color='green'); axs3[4].set_title('HSS')
    axs3[5].bar(x_labels, metrics_df['CQVSS_CQM'], color='pink'); axs3[5].set_title('CQVSS')
    fig3.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_XGB based CQR Model Experiment', fontsize=16)
    fig3.tight_layout(rect=[0, 0, 1, 0.95])
    fig3.savefig(os.path.join(out_dir, f'CQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig3)

    fig4, axs4 = plt.subplots(1, 6, figsize=(30, 6))
    axs4[0].bar(x_labels, metrics_df['Hit Rate_SQM'], color='red'); axs4[0].set_title('Hit Rate (POD)')
    axs4[1].bar(x_labels, metrics_df['False Alarm Ratio_SQM'], color='orange'); axs4[1].set_title('False Alarm Ratio (FAR)')
    axs4[2].bar(x_labels, metrics_df['Threat Score_SQM'], color='blue'); axs4[2].set_title('Threat Score (TS)')
    axs4[3].bar(x_labels, metrics_df['Bias_SQM'], color='purple'); axs4[3].set_title('Bias Score')
    axs4[4].bar(x_labels, metrics_df['Heidke Skill Score_SQM'], color='green'); axs4[4].set_title('HSS')
    axs4[5].bar(x_labels, metrics_df['SQVSS_SQM'], color='pink'); axs4[5].set_title('SQVSS')
    fig4.suptitle(f'Verification Metrics for Extreme Events (Obs > 95th Percentile = 40.29mm)_{var_comb}_SQM', fontsize=16)
    fig4.tight_layout(rect=[0, 0, 1, 0.95])
    fig4.savefig(os.path.join(out_dir, f'SQM_SkillScores_Bar_{var_comb}.png'))
    plt.close(fig4)

    # === Linear Regression ===
    fig5, ax5 = plt.subplots(figsize=(15, 6))
    r_lr, _ = pearsonr(Y_jjas, pred_lr_jjas)
    rmse_lr = np.sqrt(mean_squared_error(Y_jjas, pred_lr_jjas))
    ax5.plot(Y_jjas.index, Y_jjas, label='Observed', color='blue')
    ax5.plot(pred_lr_jjas.index, pred_lr_jjas, label='Linear Regression', color='black')
    ax5.set_title(f'Linear Regression | {var_comb} | JJAS | r={r_lr:.2f} | RMSE={rmse_lr:.2f}', fontsize=14)
    ax5.tick_params(axis='x', rotation=45)
    ax5.legend()
    # Lat-lon text
    lat_range = (22.5, 23.5)
    lon_range = (72, 73)

    # Combine with \n for a multi-line suptitle
    main_title = f'Linear Regression | {var_comb} | JJAS'
    lat_lon_text = f'Location: Lat {lat_range[0]}°–{lat_range[1]}°, Lon {lon_range[0]}°–{lon_range[1]}°'
    fig5.suptitle(f'{main_title}\n{lat_lon_text}', fontsize=16)
    fig5.tight_layout(rect=[0, 0, 1, 0.95])
    fig5.savefig(os.path.join(out_dir, f'LinearRegression_{var_comb}_JJAS.png'))
    plt.close(fig5)


C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:15:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
C:\Users\Angshudeep Majumdar\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:15:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr